# Clase 7: Taller Práctico - Otros Métodos Supervisados

**Objetivos:**

En este taller, aplicaremos y compararemos los tres métodos de clasificación que hemos estudiado en la clase teórica:

1.  **k-Nearest Neighbors (k-NN)**: Un clasificador basado en instancia y distancia.
2.  **Naive Bayes**: Un clasificador probabilístico basado en el teorema de Bayes.
3.  **Perceptrón Multicapa (MLP)**: Nuestro primer vistazo a una red neuronal simple.

Utilizaremos el conjunto de datos "Wine" de Scikit-learn, un dataset clásico para problemas de clasificación multiclase. El objetivo es clasificar vinos en una de tres categorías basándonos en sus atributos químicos.

## 1. Configuración Inicial e Importación de Librerías

In [ ]:
# Librerías para manipulación de datos
import pandas as pd
import numpy as np

import time

# Librerías de Scikit-learn para el modelado
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.decomposition import PCA

# Librerías para visualización
import plotly.express as px
import plotly.graph_objects as go

## 2. Carga y Exploración del Dataset (EDA)

El dataset "Wine" contiene los resultados de un análisis químico de vinos cultivados en la misma región en Italia pero derivados de tres cultivares diferentes. El análisis determinó las cantidades de 13 constituyentes encontrados en cada uno de los tres tipos de vinos.

In [ ]:
# Cargar el dataset
wine_data = load_wine()
X = pd.DataFrame(wine_data.data, columns=wine_data.feature_names)
y = pd.Series(wine_data.target, name='target')

# Unir características y objetivo en un solo DataFrame para exploración
df = pd.concat([X, y], axis=1)

# Mapear los números del objetivo a nombres de clases para mayor claridad
df['target_name'] = df['target'].map({0: 'Class_0', 1: 'Class_1', 2: 'Class_2'})

print("Primeras 5 filas del dataset:")
display(df.head())

print("\nDescripción estadística de las características:")
display(df.describe())

### Visualización con PCA

Dado que tenemos 13 características, no podemos visualizarlas todas a la vez. Usaremos el Análisis de Componentes Principales (PCA) para reducir la dimensionalidad a 2 componentes y visualizar la separación de las clases.

In [ ]:
# Primero, escalamos los datos para que PCA funcione correctamente
scaler_pca = StandardScaler()
X_scaled_pca = scaler_pca.fit_transform(X)

# Aplicamos PCA para reducir a 2 dimensiones
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled_pca)

# Creamos un DataFrame para la visualización
df_pca = pd.DataFrame(data=X_pca, columns=['PC1', 'PC2'])
df_pca['target_name'] = df['target_name']

# Gráfico interactivo con Plotly
fig = px.scatter(df_pca, 
                 x='PC1', 
                 y='PC2', 
                 color='target_name', 
                 title='Visualización del Dataset Wine con PCA (2 Componentes)',
                 labels={'PC1': 'Primer Componente Principal', 'PC2': 'Segundo Componente Principal'},
                 template='plotly_white')

fig.show()

La visualización PCA nos muestra que las clases están razonablemente bien separadas, aunque con algo de superposición. Esto sugiere que los algoritmos de clasificación deberían poder encontrar patrones útiles.

## 3. Preparación de los Datos para el Modelado

Ahora, dividiremos los datos en conjuntos de entrenamiento y prueba, y aplicaremos el escalado de características. El escalado es fundamental para k-NN y MLP, ya que son sensibles a las diferentes escalas de las variables.

In [ ]:
# Separar los datos originales (sin PCA) en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Inicializar el escalador
scaler = StandardScaler()

# Ajustar el escalador SÓLO con los datos de entrenamiento
X_train_scaled = scaler.fit_transform(X_train)

# Aplicar la misma transformación a los datos de prueba
X_test_scaled = scaler.transform(X_test)

print(f"Tamaño del set de entrenamiento: {X_train_scaled.shape[0]} muestras")
print(f"Tamaño del set de prueba: {X_test_scaled.shape[0]} muestras")

## 4. Implementación y Evaluación de Modelos

### Modelo 1: k-Nearest Neighbors (k-NN)

Implementaremos k-NN con un valor de `k` inicial (ej. k=5) y evaluaremos su rendimiento.

In [ ]:
start = time.time()
# Inicializar y entrenar el clasificador k-NN
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train_scaled, y_train)

# Realizar predicciones
y_pred_knn = knn.predict(X_test_scaled)

# Evaluar el modelo
print("------ Resultados de k-NN (k=5) ------")
print(f"Accuracy: {accuracy_score(y_test, y_pred_knn):.4f}")
print("\nMatriz de Confusión:")
print(confusion_matrix(y_test, y_pred_knn))
print("\nReporte de Clasificación:")
print(classification_report(y_test, y_pred_knn))
end = time.time()
print(f"{end - start:.3f} segundos para entrenar el modelo k-NN")

#### Desafío: Encontrar el `k` Óptimo

El rendimiento de k-NN depende fuertemente del valor de `k`. Un `k` muy pequeño puede llevar a sobreajuste, mientras que un `k` muy grande puede simplificar demasiado el modelo. 

**Tu tarea:** Escribe un bucle que entrene y evalúe el modelo k-NN para un rango de valores de `k` (por ejemplo, de 1 a 30). Almacena la precisión (accuracy) para cada `k` y luego crea un gráfico para visualizar cómo cambia la precisión en función de `k`. ¿Cuál es el valor de `k` que da el mejor resultado en el conjunto de prueba?

In [ ]:
# Espacio para tu solución al desafío
k_values = range(1, 31)
accuracies = []

for k in k_values:
    knn_loop = KNeighborsClassifier(n_neighbors=k)
    knn_loop.fit(X_train_scaled, y_train)
    y_pred_loop = knn_loop.predict(X_test_scaled)
    accuracies.append(accuracy_score(y_test, y_pred_loop))

# Gráfico de k vs. Accuracy
fig = go.Figure(data=go.Scatter(x=list(k_values), y=accuracies, mode='lines+markers'))
fig.update_layout(
    title='Accuracy de k-NN en función del número de vecinos (k)',
    xaxis_title='Valor de k',
    yaxis_title='Accuracy en el conjunto de prueba',
    template='plotly_white'
)
fig.show()

best_k = k_values[np.argmax(accuracies)]
print(f"\nEl valor óptimo de k es: {best_k} con una precisión de {max(accuracies):.4f}")

### Modelo 2: Naive Bayes

Ahora, aplicaremos el clasificador Naive Bayes Gaussiano. Esta variante es adecuada porque nuestras características son continuas y podemos asumir (ingenuamente) que siguen una distribución normal dentro de cada clase.

In [ ]:
start = time.time()
# Inicializar y entrenar el clasificador Naive Bayes Gaussiano
# Nota: Naive Bayes no es tan sensible al escalado, pero es buena práctica usar los datos escalados por consistencia.
gnb = GaussianNB()
gnb.fit(X_train_scaled, y_train)

# Realizar predicciones
y_pred_gnb = gnb.predict(X_test_scaled)

# Evaluar el modelo
print("------ Resultados de Naive Bayes Gaussiano ------")
print(f"Accuracy: {accuracy_score(y_test, y_pred_gnb):.4f}")
print("\nMatriz de Confusión:")
print(confusion_matrix(y_test, y_pred_gnb))
print("\nReporte de Clasificación:")
print(classification_report(y_test, y_pred_gnb))
end = time.time()
print(f"{end - start:.3f} segundos para entrenar el modelo Naive Bayes")

### Modelo 3: Perceptrón Multicapa (MLP)

Finalmente, implementaremos una red neuronal simple. Usaremos `MLPClassifier` de Scikit-learn, que es una implementación eficiente y fácil de usar. Definiremos una arquitectura simple con una capa oculta.

In [ ]:
start = time.time()
# Inicializar y entrenar el clasificador MLP
# hidden_layer_sizes=(100,) significa una capa oculta con 100 neuronas.
# max_iter=1000 para asegurar la convergencia.
mlp = MLPClassifier(hidden_layer_sizes=(100,), max_iter=1000, random_state=42)
mlp.fit(X_train_scaled, y_train)

# Realizar predicciones
y_pred_mlp = mlp.predict(X_test_scaled)

# Evaluar el modelo
print("------ Resultados del Perceptrón Multicapa (MLP) ------")
print(f"Accuracy: {accuracy_score(y_test, y_pred_mlp):.4f}")
print("\nMatriz de Confusión:")
print(confusion_matrix(y_test, y_pred_mlp))
print("\nReporte de Clasificación:")
print(classification_report(y_test, y_pred_mlp))
end = time.time()
print(f"{end - start:.3f} segundos para entrenar el modelo MLP")

## 5. Comparación y Conclusiones

En este caso particular, los tres modelos han obtenido una precisión muy alta, a menudo perfecta o casi perfecta en el conjunto de prueba. Esto se debe a que el dataset Wine es un problema relativamente "fácil" con clases bien separadas.

* **k-NN** demostró ser muy efectivo, especialmente después de encontrar un buen valor para `k`.
* **Naive Bayes** también funcionó excepcionalmente bien, lo que sugiere que, para este problema, el supuesto de independencia condicional no fue una limitación grave.
* El **MLP** igualó el rendimiento de los otros modelos, mostrando su capacidad para resolver problemas de clasificación, aunque su entrenamiento es computacionalmente más costoso.

En problemas del mundo real más complejos y con mayor superposición entre clases, las diferencias en el rendimiento de estos algoritmos suelen ser más pronunciadas. La elección del modelo dependerá de las características específicas del problema, la cantidad de datos disponibles y los recursos computacionales.

## 6. Ejercicios Adicionales

Para solidificar tu comprensión, aquí tienes 10 ejercicios para explorar por tu cuenta.

1.  **Cambiar el Dataset:** Carga el dataset `load_breast_cancer` de `sklearn.datasets`. Es un problema de clasificación binaria. Repite el análisis completo (EDA, preprocesamiento, modelado y comparación) para este nuevo dataset. ¿Qué modelo funciona mejor aquí?

2. **Arquitectura del MLP:** Experimenta con diferentes arquitecturas para el `MLPClassifier`. Prueba con más capas ocultas (ej. `hidden_layer_sizes=(100, 50,)`) o con diferente número de neuronas por capa. ¿Puedes mejorar la precisión obtenida?

3. **Métricas de Distancia en k-NN:** El `KNeighborsClassifier` tiene un parámetro `metric`. Por defecto es `'minkowski'` (que con p=2 es la distancia Euclidiana). Prueba a cambiarlo a `'manhattan'` (Distancia de Manhattan). ¿Cambia el rendimiento del modelo?

4. **Robustez del `train_test_split`:** Cambia el valor de `random_state` en la función `train_test_split`. Repite el entrenamiento y evaluación de los tres modelos. ¿Son los resultados de precisión exactamente los mismos? ¿Qué nos dice esto sobre la evaluación de modelos?

5. **Comparar con Regresión Logística:** Como un modelo de base adicional, importa `LogisticRegression` de `sklearn.linear_model`. Entrénalo y evalúalo en el dataset Wine. ¿Cómo se compara su rendimiento con los tres modelos de esta clase?